# https://sunrise-sunset.org/search?location=Thunder+Bay

In [ ]:
from math import *
from geopy.geocoders import Nominatim
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
import json
import requests
from datetime import datetime
import pandas as pd         

In [85]:
#solar radiation
# lat = latitude
# n = number of day
# t = time
# to = real midday
# k = scale of cloudiness
# a = inclination angle of the pannel
def solar_radiation(lat,n,t,to,k,a):
    delta = 23.5*sin(radians(360*(284+n)/365))
    h = degrees(asin(cos(radians(lat))*cos(radians(delta))*cos(radians(15*(t-to))) + sin(radians(lat))*sin(radians(delta))))
    s = 1
    qo = (0.62+0.68)*sin(radians(h))
    A = degrees(asin((cos(radians(delta))*sin(radians(15*(t-to))))/cos(radians(h))))
    qr = qo*(1-0.38*(1+(k/10))*(k/10))
    q = qr*(cos(radians(a))*sin(radians(h)) + (sin(radians(a))*cos(radians(h))*cos(radians(A))))/sin(radians(h))
    return q
def pannel_area(n=1):
    length = 65*0.0254
    width = 39*0.0254
    return length*width*n

def solar_power(radiation,ambient_temp,panels=1,eff=0.2):
    efficiency = eff
    panel_area = pannel_area(panels)
    power = efficiency*panel_area*radiation*(1-0.005*(ambient_temp-25))
    return power
#generate time
def generate_time():
    time = []
    for i in range(240):
        time.append(i/10)
    return time

#get longitude and latitude
def latitude(address):
    geolocator = Nominatim(user_agent="my_user_agent")
    city ="Thunder Bay"
    country ="Canada"
    loc = geolocator.geocode(address+','+city+','+ country)
    return loc.latitude
def longitude(address):
    geolocator = Nominatim(user_agent="my_user_agent")
    city ="Thunder Bay"
    country ="Canada"
    loc = geolocator.geocode(address+','+city+','+ country)
    return loc.longitude

def get_day_number(date):
    days = [31,28,31,30,31,30,31,31,30,31,30,31]
    date = date.split("-")
    month = int(date[1]);
    res = sum(days[:month-1])
    res = res + int(date[2])
    return res

def solar_noon(date,address):
    response = requests.get("https://api.sunrise-sunset.org/json?lat="+str(latitude(address)) + "&lng="+str(longitude(address))+"&date="+date)
    result = response.json()['results']['solar_noon'].split(' ')
    time = result[0].split(":")
    hour = int(time[0])
    if(hour<12 and result[1]=="PM"):
        hour = hour + 12
    hour = hour-5
    minute = round(int(time[1])/6)
    return hour+(minute/10)
def get_weather(w,date):
    x = w[w['Date']==date].index.tolist()
    temp = []
    for i in x:
        temp.append(w.iloc[i]['Temp (°C)'])
    x = np.arange(0,240,1)
    l = []
    t= 0;
    for i in range(len(x)):
        if(i%10==0):
            l.append(temp[t])
            t = t+1
        else:
            l.append(np.nan)
    p = pd.DataFrame(l)
    p = p.interpolate(method ='linear', limit_direction ='both')
    temp = p[0]
    return temp


def get_solar_radiation(date,address):
    lat = latitude(address)
    to = solar_noon(date,address)
    n = get_day_number(date)
    k = 1
    a = 22.5
    t = generate_time()
    pred = []
    for i in t:
        res = solar_radiation(lat,n,i,to,k,a)
        if(res>0):
            pred.append(res)
        else:
            pred.append(0)
    return t,pred

def get_solar_power(temp,radiation,panels=1,eff=0.2):
    power = []
    for i in range(len(temp)):
        power.append(solar_power(radiation[i],temp[i],panels,eff))
    return power
def visualize(x,y):
    fig = plt.figure()
    fig.set_size_inches(9.5, 5.5)
    ax = plt.axes()
    ax.plot(x, y);
def gen_hourly_data(df,date,address):
    response = requests.get("https://api.sunrise-sunset.org/json?lat="+str(latitude(address)) + "&lng="+str(longitude(address))+"&date="+date)
    sunrise = int(utc_to_est(response.json()['results']['sunrise'])*10)
    sunset = int(utc_to_est(response.json()['results']['sunset'],True)*10)
    
    power = list(df['Power'])
    temp = power[sunrise:sunset]
    print(sunrise,sunset)
    final = []
    for i in power:
        if(i in temp):
            final.append(i)
        else:
            final.append(0)
    power = final
#     print(final)
    hourly = []
    for i in range(24):
        res = sum(power[10*i:10*(i+1)])
        hourly.append(res)
    return hourly
def gen_solar(date,address,panels,eff):
    w = pd.read_csv('Weather.csv')
    temp = get_weather(w,date)
    t,pred = get_solar_radiation(date,address)
    power = get_solar_power(temp,pred,panels,eff)
    #     visualize(t,pred)
    df = pd.DataFrame(t,columns =['Time'])
    df['Radiation'] = pred
    df['Power'] = power
    df = df.set_index(df.Time)
    df = df.drop('Time',axis=1)
#     df.plot()
    df = gen_hourly_data(df,date,address)
    #crop time based on sunrise and sunset
    return df
def utc_to_est(time,sunset=False):
    print(time)
    if(sunset==False):
        hour = int(time.split(":")[0])
        minute = int(int(time.split(":")[1])/6)
        ampm = time.split(" ")[1]
        if(hour<12 and ampm=='PM'):
            hour = hour+12-5
    else:
        hour = int(time.split(":")[0])
        minute = int(int(time.split(":")[1])/6)
        ampm = time.split(" ")[1]
        if(hour<12 and ampm=='AM'):
            hour = hour+24-5
        else:
            hour = hour+12-5
    return hour+ minute*0.1

In [ ]:
response = requests.get("https://api.sunrise-sunset.org/json?lat="+str(latitude()) + "&lng="+str(longitude())+"&date="+'2016-02-12')
sunrise = utc_to_est(response.json()['results']['sunrise'])
sunrise

panel efficiency*panel area* radiation_value*(1-0.005(ambient_temperature-25 degree Celsius)).

In [86]:
# w = pd.read_csv('Weather.csv')
# for i in range(1):
#     date = "2014-" +str(i+1)+"-12"
#     temp = get_weather(w,date)
#     t,pred = get_solar_radiation(date)
#     power = get_solar_power(temp,pred)
# #     visualize(t,pred)
# df = pd.DataFrame(t,columns =['Time'])
# df['Radiation'] = pred
# df['Power'] = power
# df = df.set_index(df.Time)
# df = df.drop('Time',axis=1)
# df.plot()
address = '557 Beverly St'
h1 = gen_solar("2016-6-12",address,3,0.1)
# h2 = gen_solar("2016-6-12",2,0.075)
# data = pd.DataFrame([h1],columns=['h1'])
h1

9:54:57 AM
1:59:15 AM
99 209


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.4756873281814776,
 5.227958664787977,
 5.87999407592542,
 6.216959833710246,
 6.20771228668741,
 5.8949075195800384,
 5.2834955243845085,
 4.363808494413365,
 3.2715126878090395,
 2.6640564993122324,
 2.166446082899453,
 1.5378614891531193,
 0,
 0,
 0]

In [ ]:
plt.plot(h1)

In [ ]:
geolocator = Nominatim(user_agent="my_user_agent")
street = "553 Beverly St"
city ="Thunder Bay"
country ="Canada"
loc = geolocator.geocode(street+','+city+','+ country)
loc.latitude,loc.longitude

In [ ]:
geolocator = Nominatim(user_agent="my_user_agent")
street = "136 Greenpark Crescent"
city ="Thunder Bay"
country ="Canada"
loc = geolocator.geocode(street+','+city+','+ country)
loc.latitude,loc.longitude

In [ ]:
date = []
days = [31,29,31,30,31,30,31,31,30,31,30,31]
for i in range(12):
    for j in range(days[i]):
        date.append("2016-"+str(i+1)+"-"+str(j+1))
power = []
for i in range(31):
    power.append(sum(gen_solar(date[i])))
    print(i+1)
power

In [ ]:
prod = [41.512471979217196,
 41.32612748900243,
 41.333485178633055,
 42.29716427647197,
 41.020814537202064,
 41.434686523778005,
 41.475824427588684,
 41.22158507869385,
 43.434732915325185,
 44.69614943247117,
 44.967842055928216,
 45.51706732734248,
 45.4403128758042,
 45.12918222744948,
 43.95934198798366,
 45.04144283147737,
 46.53156825117986,
 46.54611007090597,
 46.10631315782169,
 45.08309051749658,
 45.098984580543586,
 45.830755833711464,
 44.68426248211288,
 44.86604324495846,
 45.18253839875453,
 45.65364109972812,
 45.76910278196725,
 47.21815787339588,
 48.31808128984668,
 45.05705295154496,
 45.572504011352684]

cons = [21.978670701916666,
 27.771383573249995,
 20.803626811233332,
 18.337463083299998,
 18.930406185724994,
 17.247675253875,
 14.95412531323333,
 20.095171809641663,
 20.025818171999997,
 20.917858461741666,
 18.876140633291666,
 19.970526644699998,
 19.965857153566663,
 16.53836958385833,
 13.476971685849998,
 19.206537169399997,
 15.781159596724999,
 19.93739403165833,
 15.832845604358333,
 15.177668954941666,
 17.56791293458333,
 15.340652552749997,
 24.531142043849997,
 19.597972085083327,
 13.070038588149997,
 16.678541175266666,
 15.170770590433333,
 15.942892998025004,
 13.782087733258335,
 26.58856749519167,
 19.873670080424997]

temp = pd.DataFrame(cons,columns=['Consumption'])
temp['Production'] = prod
temp.plot()